In [10]:
import anndata
import scipy.sparse as sp
import numpy as np
import scanpy as sc
import torch
from torch.utils.data.dataset import Dataset

In [19]:
class SingleCellData(Dataset):
    def __init__(self, data_path, num_gene, normalized=True):
        self.data_path = data_path
        self.num_gene = num_gene
        self.normalized = normalized
        self.anndata = anndata.read_h5ad(data_path)
        sc.pp.filter_genes(self.anndata, min_counts=1)
        self.gene_mask = self.select_gene(data=self.anndata.X, num_gene=self.num_gene)
        
        if self.normalized:
            anndata_norm = self.anndata.copy()
            sc.pp.normalize_per_cell(anndata_norm, counts_per_cell_after=1_000_000)
            sc.pp.log1p(anndata_norm)
            anndata_norm.X = anndata_norm.X.toarray()
            anndata_norm.X -= anndata_norm.X.mean(axis=0)
            anndata_norm.X /= anndata_norm.X.std(axis=0)
            self.anndata_preprocessed = anndata_norm[:, self.gene_mask].copy()
        else:
            self.anndata_preprocessed = self.anndata.copy()
            
        self.X = torch.tensor(self.anndata_preprocessed.X)
        self.id_to_batch, self.batch_to_id = self.get_batch_map()
        self.id_to_cell, self.cell_to_id = self.get_cell_map()
        
        
        self.cell_label_tensor = torch.tensor([self.cell_to_id[e] for e in self.anndata.obs['labels']])
        self.batch_id_tensor = torch.tensor([self.batch_to_id[e] for e in self.anndata.obs['batch_id']])
        
    def __getitem__(self, index):
        return self.X[index], self.cell_label_tensor[index], self.batch_id_tensor[index]
            
    def __len__(self):
        return len(self.anndata)
    
    def get_batch_map(self):
        num_batch_type = len(self.anndata.obs['batch_id'].unique().tolist())
        id2batch = dict(zip(list(range(num_batch_type)), self.anndata.obs['batch_id'].unique().tolist()))
        batch2id = {v: k for k,v in id2batch.items()}
        return id2batch, batch2id
    
    def get_cell_map(self):
        num_cell_type = len(self.anndata.obs['labels'].unique().tolist())
        id2cell = dict(zip(list(range(num_cell_type)), self.anndata.obs['labels'].unique().tolist()))
        cell2id = {v:k for k,v in id2cell.items()}
        return id2cell, cell2id
    
    def select_gene(self, data,\
                    num_gene,\
                    threshold=0,\
                    atleast=10,\
                    decay=1,
                    xoffset=5,\
                    yoffset=0.02):
        
        if sp.issparse(data):
            zeroRate = 1 - np.squeeze(np.array((data > threshold).mean(axis=0)))
            A = data.multiply(data > threshold)
            A.data = np.log2(A.data)
            meanExpr = np.zeros_like(zeroRate) * np.nan
            detected = zeroRate < 1
            meanExpr[detected] = np.squeeze(np.array(A[:, detected].mean(axis=0))) / (
                1 - zeroRate[detected]
            )
        else:
            zeroRate = 1 - np.mean(data > threshold, axis=0)
            meanExpr = np.zeros_like(zeroRate) * np.nan
            detected = zeroRate < 1
            meanExpr[detected] = np.nanmean(
                np.where(data[:, detected] > threshold, np.log2(data[:, detected]), np.nan),
                axis=0,
            )

        lowDetection = np.array(np.sum(data > threshold, axis=0)).squeeze() < atleast
        # lowDetection = (1 - zeroRate) * data.shape[0] < atleast - .00001
        zeroRate[lowDetection] = np.nan
        meanExpr[lowDetection] = np.nan

        if self.num_gene is not None:
            up = 10
            low = 0
            for t in range(100):
                nonan = ~np.isnan(zeroRate)
                selected = np.zeros_like(zeroRate).astype(bool)
                selected[nonan] = (
                    zeroRate[nonan] > np.exp(-decay * (meanExpr[nonan] - xoffset)) + yoffset
                )
                if np.sum(selected) == num_gene:
                    break
                elif np.sum(selected) < num_gene:
                    up = xoffset
                    xoffset = (xoffset + low) / 2
                else:
                    low = xoffset
                    xoffset = (xoffset + up) / 2
            print("Chosen offset: {:.2f}".format(xoffset))
        else:
            nonan = ~np.isnan(zeroRate)
            selected = np.zeros_like(zeroRate).astype(bool)
            selected[nonan] = (
                zeroRate[nonan] > np.exp(-decay * (meanExpr[nonan] - xoffset)) + yoffset
            )

        return selected
    

In [20]:
sc_dataset = SingleCellData(data_path="./data/baron_2016h.h5ad", num_gene=3000)

Chosen offset: 0.18


In [48]:
x, y, b = sc_dataset.__getitem__(100)

In [49]:
x,y,b

(tensor([-0.1800, -0.2322, -0.2284,  ..., -0.2531, -0.6688, -0.3976]),
 tensor(2),
 tensor(0))

In [50]:
x[0:100]

tensor([-0.1800, -0.2322, -0.2284, -0.0547, -0.2252, -0.0515, -0.0607,  3.8628,
        -0.3273,  1.3579, -0.0373, -0.3545, -0.1263, -0.0921, -0.1806, -0.4243,
        -0.0524, -0.0902, -0.2024, -0.2693, -0.1517, -0.0782, -0.1185, -0.1924,
        -0.2157, -0.0780,  1.7218, -0.1622, -0.1276, -0.1960, -0.1290, -0.1898,
        -0.1390, -0.1141, -0.0952, -0.0755, -0.1004, -0.5188, -0.0646, -0.2540,
        -0.1685, -0.0492, -0.1760, -0.2406, -0.1463, -0.1237, -0.2599,  2.4256,
        -0.0387, -0.2471, -0.0769, -0.2236, -0.2255, -0.2282, -0.5694, -0.0373,
        -0.0711,  2.0229, -0.1197, -0.2199, -0.0635, -0.3017, -0.1212, -0.0579,
        -0.1760, -0.7531, -0.0681, -0.2296, -0.1419, -0.0443, -0.1388, -0.0559,
        -0.2289, -0.6069, -0.1159, -0.0535, -0.1717, -0.0468, -0.1249, -0.0886,
        -0.2274, -0.2573, -0.0994, -0.0749, -0.0697, -0.4340, -0.5196, -0.1195,
        -0.5298, -0.0735, -0.4232, -0.1335, -0.3099, -0.0798, -0.1449, -0.1025,
        -0.1506, -0.3007, -0.0994, -0.31

In [37]:
sc_dataset.cell_to_id

{'Acinar cells': 0,
 'Beta cells': 1,
 'Delta cells': 2,
 'PaSC': 3,
 'Ductal cells': 4,
 'Alpha cells': 5,
 'Other': 6,
 'PP cells': 7,
 'Endothelial cell': 8}

In [38]:
sc_dataset.batch_to_id

{'human1_lib1': 0,
 'human1_lib2': 1,
 'human1_lib3': 2,
 'human2_lib1': 3,
 'human2_lib2': 4,
 'human2_lib3': 5,
 'human3_lib1': 6,
 'human3_lib2': 7,
 'human3_lib3': 8,
 'human3_lib4': 9,
 'human4_lib1': 10,
 'human4_lib3': 11}

In [42]:
adata = anndata.read_h5ad("./data/baron_2016h.h5ad")
sc.pp.filter_genes(adata, min_counts=1)
adata

AnnData object with n_obs × n_vars = 8569 × 17499 
    obs: 'batch_id', 'labels'
    var: 'n_counts'
    uns: 'name', 'organism', 'tissue', 'year'

In [43]:
def select_genes(
    data,
    threshold=0,
    atleast=10,
    yoffset=0.02,
    xoffset=5,
    decay=1,
    n=None,
    alpha=1,
):
    if sp.issparse(data):
        zeroRate = 1 - np.squeeze(np.array((data > threshold).mean(axis=0)))
        A = data.multiply(data > threshold)
        A.data = np.log2(A.data)
        meanExpr = np.zeros_like(zeroRate) * np.nan
        detected = zeroRate < 1
        meanExpr[detected] = np.squeeze(np.array(A[:, detected].mean(axis=0))) / (
            1 - zeroRate[detected]
        )
    else:
        zeroRate = 1 - np.mean(data > threshold, axis=0)
        meanExpr = np.zeros_like(zeroRate) * np.nan
        detected = zeroRate < 1
        meanExpr[detected] = np.nanmean(
            np.where(data[:, detected] > threshold, np.log2(data[:, detected]), np.nan),
            axis=0,
        )

    lowDetection = np.array(np.sum(data > threshold, axis=0)).squeeze() < atleast
    # lowDetection = (1 - zeroRate) * data.shape[0] < atleast - .00001
    zeroRate[lowDetection] = np.nan
    meanExpr[lowDetection] = np.nan

    if n is not None:
        up = 10
        low = 0
        for t in range(100):
            nonan = ~np.isnan(zeroRate)
            selected = np.zeros_like(zeroRate).astype(bool)
            selected[nonan] = (
                zeroRate[nonan] > np.exp(-decay * (meanExpr[nonan] - xoffset)) + yoffset
            )
            if np.sum(selected) == n:
                break
            elif np.sum(selected) < n:
                up = xoffset
                xoffset = (xoffset + low) / 2
            else:
                low = xoffset
                xoffset = (xoffset + up) / 2
        print("Chosen offset: {:.2f}".format(xoffset))
    else:
        nonan = ~np.isnan(zeroRate)
        selected = np.zeros_like(zeroRate).astype(bool)
        selected[nonan] = (
            zeroRate[nonan] > np.exp(-decay * (meanExpr[nonan] - xoffset)) + yoffset
        )

    return selected

In [44]:
gene_mask = select_genes(adata.X, n=3000, threshold=0)

adata_norm = adata.copy()
sc.pp.normalize_per_cell(adata_norm, counts_per_cell_after=1_000_000)
sc.pp.log1p(adata_norm)

Chosen offset: 0.18


In [45]:
adata_norm.X = adata_norm.X.toarray()
adata_norm.X -= adata_norm.X.mean(axis=0)
adata_norm.X /= adata_norm.X.std(axis=0)

In [46]:

adata_3000 = adata_norm[:, gene_mask].copy()

In [62]:
adata_3000.obs['batch_id'][4578]

'human3_lib2'

In [63]:
adata_3000.obs['labels'][4578]

'Acinar cells'

In [65]:
x,y,b = sc_dataset.__getitem__(4578)

In [70]:
adata_3000.X[4578]

array([-0.18004292, -0.23224501, -0.22836281, ..., -0.25314525,
        0.836425  ,  2.0954144 ], dtype=float32)

In [73]:
np.array_equal(x,adata_3000.X[4578])

True

In [66]:
b

tensor(7)

In [67]:
y

tensor(0)

In [61]:
sc_dataset.id_to_cell

{0: 'Acinar cells',
 1: 'Beta cells',
 2: 'Delta cells',
 3: 'PaSC',
 4: 'Ductal cells',
 5: 'Alpha cells',
 6: 'Other',
 7: 'PP cells',
 8: 'Endothelial cell'}

In [68]:
sc_dataset.id_to_batch

{0: 'human1_lib1',
 1: 'human1_lib2',
 2: 'human1_lib3',
 3: 'human2_lib1',
 4: 'human2_lib2',
 5: 'human2_lib3',
 6: 'human3_lib1',
 7: 'human3_lib2',
 8: 'human3_lib3',
 9: 'human3_lib4',
 10: 'human4_lib1',
 11: 'human4_lib3'}